In [1]:
import numpy as np
from scipy.fftpack import fftfreq
from scipy.interpolate import RegularGridInterpolator
import matplotlib
matplotlib.use('Qt5Agg') 
import matplotlib.pyplot as plt
from tqdm import tqdm

# Constants
N_POINTS = 250
KINEMATIC_VISCOSITY = 0.0001
TIME_STEP_LENGTH = 0.01
N_TIME_STEPS = 300

def backtrace(backtraced_positions, original_positions, direction):
    """
    Perform an Euler step backward in time for self-advection and enforce periodic boundary conditions.
    """
    # Euler step backward and wrap positions into [0.0, 1.0)
    backtraced_positions[:] = np.mod(
        original_positions - TIME_STEP_LENGTH * direction,
        1.0,
    )

def interpolate_positions(field_interpolated, field, interval_x, interval_y, query_points_x, query_points_y):
    """
    Interpolate the field at the backtraced positions using linear interpolation.
    """
    # Create a regular grid interpolator
    interpolator = RegularGridInterpolator(
        (interval_x, interval_y),
        field,
        method='linear',
        bounds_error=False,
        fill_value=None
    )
    # Stack the query points and evaluate the interpolator
    points = np.stack((query_points_x.flatten(), query_points_y.flatten()), axis=-1)
    field_interpolated[:] = interpolator(points).reshape(field_interpolated.shape)

def main():
    # Create spatial grid
    element_length = 1.0 / (N_POINTS - 1)
    x_interval = np.linspace(0.0, 1.0, N_POINTS)
    y_interval = np.linspace(0.0, 1.0, N_POINTS)
    coordinates_x, coordinates_y = np.meshgrid(x_interval, y_interval, indexing='ij')

    # Compute wavenumbers for Fourier transforms
    wavenumbers_1d = fftfreq(N_POINTS) * N_POINTS
    wavenumbers_x, wavenumbers_y = np.meshgrid(wavenumbers_1d, wavenumbers_1d, indexing='ij')
    wavenumbers_norm = np.sqrt(wavenumbers_x**2 + wavenumbers_y**2)
    decay = np.exp(- TIME_STEP_LENGTH * KINEMATIC_VISCOSITY * wavenumbers_norm**2)

    # Avoid division by zero in normalization
    wavenumbers_norm[wavenumbers_norm == 0] = 1.0
    normalized_wavenumbers_x = wavenumbers_x / wavenumbers_norm
    normalized_wavenumbers_y = wavenumbers_y / wavenumbers_norm

    # Define the external forces (two opposing Gaussian forces)
    force_x = 100.0 * (
        np.exp(- 1.0 / (2 * 0.005) * ((coordinates_x - 0.2)**2 + (coordinates_y - 0.45)**2))
        -
        np.exp(- 1.0 / (2 * 0.005) * ((coordinates_x - 0.8)**2 + (coordinates_y - 0.55)**2))
    )

    # Initialize arrays
    backtraced_coordinates_x = np.zeros_like(coordinates_x)
    backtraced_coordinates_y = np.zeros_like(coordinates_y)
    velocity_x = np.zeros_like(coordinates_x)
    velocity_y = np.zeros_like(coordinates_y)
    velocity_x_prev = np.zeros_like(velocity_x)
    velocity_y_prev = np.zeros_like(velocity_y)
    # For FFT operations, use complex data types
    velocity_x_fft = np.zeros_like(velocity_x, dtype=np.complex128)
    velocity_y_fft = np.zeros_like(velocity_y, dtype=np.complex128)
    pressure_fft = np.zeros_like(coordinates_x, dtype=np.complex128)

    # Set up plotting
    plt.figure(figsize=(8, 6))

    # Time-stepping loop
    for iter in tqdm(range(N_TIME_STEPS), desc="Time-stepping"):
        # (1) Apply the forces
        time_current = iter * TIME_STEP_LENGTH
        pre_factor = max(1 - time_current, 0.0)
        velocity_x_prev += TIME_STEP_LENGTH * pre_factor * force_x

        # (2) Self-advection by backtracing and interpolation
        backtrace(backtraced_coordinates_x, coordinates_x, velocity_x_prev)
        backtrace(backtraced_coordinates_y, coordinates_y, velocity_y_prev)
        interpolate_positions(
            velocity_x, velocity_x_prev, x_interval, y_interval,
            backtraced_coordinates_x, backtraced_coordinates_y
        )
        interpolate_positions(
            velocity_y, velocity_y_prev, x_interval, y_interval,
            backtraced_coordinates_x, backtraced_coordinates_y
        )

        # (3.1) Transform velocities into Fourier domain
        velocity_x_fft = np.fft.fft2(velocity_x)
        velocity_y_fft = np.fft.fft2(velocity_y)

        # (3.2) Diffuse velocities by applying decay in Fourier space
        velocity_x_fft *= decay
        velocity_y_fft *= decay

        # (3.3) Compute pseudo-pressure in Fourier domain
        pressure_fft = (
            velocity_x_fft * normalized_wavenumbers_x +
            velocity_y_fft * normalized_wavenumbers_y
        )

        # (3.4) Project velocities to be incompressible
        velocity_x_fft -= pressure_fft * normalized_wavenumbers_x
        velocity_y_fft -= pressure_fft * normalized_wavenumbers_y

        # (3.5) Transform back to spatial domain
        velocity_x = np.real(np.fft.ifft2(velocity_x_fft))
        velocity_y = np.real(np.fft.ifft2(velocity_y_fft))

        # Update previous velocities for the next iteration
        velocity_x_prev[:] = velocity_x
        velocity_y_prev[:] = velocity_y

        # Visualization of the vorticity (curl of the velocity field)
        d_u__d_y = np.diff(velocity_x, axis=1)[1:, :]
        d_v__d_x = np.diff(velocity_y, axis=0)[:, 1:]
        curl = d_u__d_y - d_v__d_x

        plt.clf()
        plt.imshow(
            curl.T,
            origin='lower',
            extent=[0, 1, 0, 1],
            cmap='seismic',
            aspect='equal'
        )
        plt.title(f"Fluid Flow Simulation at Time Step {iter+1}")
        plt.colorbar(label='Vorticity')
        plt.pause(0.001)

    plt.show()

if __name__ == "__main__":
    main()


Time-stepping:   0%|          | 1/300 [00:00<00:44,  6.73it/s]2024-12-10 19:27:03.038 Python[96604:2031586] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-10 19:27:03.038 Python[96604:2031586] +[IMKInputSession subclass]: chose IMKInputSession_Modern
Time-stepping: 100%|██████████| 300/300 [00:18<00:00, 16.15it/s]
